In [ ]:
import itertools
import traceback

import networkx as nx
import numpy as np
import pandas as pd
import pyomo.environ as pe

# Data Synthesis

First, we construct a random graph. The specified seed yields a fully connected graph.

In [ ]:
N = 6
p = 0.50
graph = nx.erdos_renyi_graph(N, p, seed=42, directed=True)
nx.draw_networkx(graph)

In [ ]:
data = {}
np.random.seed(100)
for edge in graph.edges():
    data[edge] = {'weight': np.random.geometric(0.5)}

In [ ]:
df = pd.DataFrame.from_dict(data, orient='index')
df.index.names = ['i', 'j']
display(df)

In [ ]:
df_cost = df['weight'].reset_index()\
                      .pivot(index='i', columns='j', values='weight')\
                    .  reindex(index=graph.nodes(), columns=graph.nodes())
display(df_cost)

# Model Setup

For the sake of demonstration, we purposely built a graph that is not complete. That way we are forced to consider how to build the edge set (which in a complete graph is just the Cartesian product of the node set with itself, perhaps excluding self-loops).

### Attempt 1: Workable, but error prone.

In [ ]:
nodes = set(itertools.chain(*df.index))

model = pe.ConcreteModel()
model.nodes = pe.Set(initialize=nodes)
# initialize as Cartesian product of node set with itself
model.edges = pe.Set(initialize=model.nodes*model.nodes)

In [ ]:
len(model.edges)

### Attempt 2: Incorrect.

In [ ]:
nodes = set(itertools.chain(*df.index))

model = pe.ConcreteModel()
model.nodes = pe.Set(initialize=nodes)
# declare as a subset of the Cartesian product
model.edges = pe.Set(within=model.nodes*model.nodes)

In [ ]:
len(model.edges)

### Attempt 3: Correct and more robust than Attempt #1.

In [ ]:
nodes = set(itertools.chain(*df.index))
edges = set(df.index)

model = pe.ConcreteModel()
model.nodes = pe.Set(initialize=nodes)
# declare as a subset of the Cartesian product, initialize as DataFrame index
model.edges = pe.Set(within=model.nodes*model.nodes, initialize=edges)

In [ ]:
len(model.edges)

## Why `within`? Free error checking.

Here, we see that specifying the `within` keyword for the edge set helps to curate our inputs. To show this, we purposely try to add an edge that is _not_ in the Cartesian product of the node set with itself. Pyomo correctly detects the error and raises an error.

In [ ]:
nodes = set(itertools.chain(*df.index))
edges = set(df.index.tolist() + [(0, 1234)])

model = pe.ConcreteModel()
model.nodes = pe.Set(initialize=nodes)

try:
    model.edges = pe.Set(within=model.nodes*model.nodes, initialize=edges)
except ValueError:
    traceback.print_exc()

Without the `within` keyword, Pyomo will accept virtually any immutable object.

In [ ]:
nodes = set(itertools.chain(*df.index))
edges = set(df.index.tolist() + [('foo', 'bar')])

model = pe.ConcreteModel()
model.nodes = pe.Set(initialize=nodes)
model.edges = pe.Set(initialize=edges) # no `within` keyword, no sanity check!

## Benefits of Importing `dict`-like Data

In [ ]:
nodes = set(itertools.chain(*df.index))

model = pe.ConcreteModel()
model.nodes = pe.Set(initialize=nodes)
model.edges = pe.Set(within=model.nodes*model.nodes, initialize=df.index)
# build parameter from table data (nodes * nodes)
model.cost = pe.Param(model.edges, initialize=df_cost.stack().to_dict())

In [ ]:
df_cost.stack().to_dict()

In [ ]:
nodes = set(itertools.chain(*df.index))

model = pe.ConcreteModel()
model.nodes = pe.Set(initialize=nodes)
model.edges = pe.Set(within=model.nodes*model.nodes, initialize=df.index)
# build parameter from flattened data indexed by edge set
model.cost = pe.Param(model.edges, initialize=df['weight'].to_dict())

In [ ]:
df['weight'].to_dict()

In [ ]:
dict(model.cost)